<div class="alert alert-info">
    <h1> About these notebooks </h1>
    <p> If you opened this notebook in Binder, it is running on a server that was launched <b>just</b> for you. Your changes will be reset once server restarts due to inactivity, so don't rely on it for anything you want to last. Likewise, feel free to try and tweak everything you want, since you won't affect the original repository. And <b>if you have never used a Jupyter noteboook: </b> for running a cell just press <tt>Shift + Enter.</tt> </p>
    <p>Enjoy!</p>
</div>

In [1]:
Threads.nthreads()

1

In [54]:
include("GameOfLife.jl")

""" Game of Life iteration for the zero boundary condition case"""
function game_it_zero_bdry_parallel(A)
    M, N = size(A)
    B = zeros(Int64, M, N)
    Threads.@threads for j in 2:N-1
        for i in 2:M-1
            sum_neigh = A[i-1,j-1] + A[i-1,j] + A[i-1,j+1] + A[i,j-1] + A[i,j+1] + A[i+1,j-1] + A[i+1,j] + A[i+1,j+1]
            if sum_neigh == 3 || (sum_neigh == 2 && A[i,j] == 1)
                B[i,j] = 1
            end
        end
    end
    return B
end


game_it_zero_bdry_parallel

In [53]:
a = zeros(10,10)

Threads.@threads for i = 1:10
    for j in 1:10
           a[j,i] = Threads.threadid()
       end
end
a

10×10 Array{Float64,2}:
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0
 1.0  1.0  1.0  2.0  2.0  2.0  3.0  3.0  4.0  4.0

In [68]:
function check_implementation()
    for i in 1:100
        A = rand([0,1], 100,100)
        @assert game_it_zero_bdry(A) == game_it_zero_bdry_parallel(A)
    end
    println("pass")
end

check_implementation()

pass


In [66]:
using BenchmarkTools: @benchmark

A = rand([0,1],100,100)
@benchmark game_it_zero_bdry(A)

BenchmarkTools.Trial: 
  memory estimate:  78.20 KiB
  allocs estimate:  2
  --------------
  minimum time:     51.850 μs (0.00% GC)
  median time:      56.180 μs (0.00% GC)
  mean time:        59.376 μs (3.31% GC)
  maximum time:     1.320 ms (94.50% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [67]:
@benchmark game_it_zero_bdry_parallel(A)

BenchmarkTools.Trial: 
  memory estimate:  81.23 KiB
  allocs estimate:  32
  --------------
  minimum time:     28.398 μs (0.00% GC)
  median time:      36.075 μs (0.00% GC)
  mean time:        39.486 μs (6.62% GC)
  maximum time:     1.637 ms (94.41% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [51]:
my_game_it_parallel(A)

10×10 Array{Int64,2}:
 0  0  0  0  0  0  1  0  0  1
 1  1  0  1  0  0  1  1  1  0
 1  0  0  1  0  1  0  0  0  0
 1  0  0  0  1  0  1  1  0  0
 1  0  0  1  1  1  1  1  0  0
 0  0  1  0  1  1  0  0  1  1
 0  0  1  1  0  0  1  1  0  0
 1  1  0  1  1  1  1  0  1  0
 1  0  1  0  1  1  1  1  0  1
 0  0  0  0  0  0  0  0  0  0

In [70]:
Threads.nthreads()

4